해당문서에 있는 내용
- 1. Binance data api 로 받아오기
- 2. 예전에 만들었던 업비트 api함수로 데이터 받아오기

In [157]:
import ccxt
import pandas as pd
from datetime import datetime
# pip install finance-datareader 패키지 이름 주의
import FinanceDataReader as fdr

In [158]:
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds() # 현재시간?
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe) # 초단위로 바꿈
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    if isinstance(since, str):
        since = exchange.parse8601(since)

    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        if len(all_ohlcv) % 100000 == 0:
            print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')

In [160]:
binance = ccxt.binance()
all_ohlcv = scrape_ohlcv(binance, 3, 'XRP/USDT', '1m', '2017-01-01 00:00:00Z', 1000)

100000 XRP/USDT candles in total from 2021-09-21T23:40:00.000Z to 2021-11-30T10:19:00.000Z
200000 XRP/USDT candles in total from 2021-07-14T13:00:00.000Z to 2021-11-30T10:19:00.000Z
300000 XRP/USDT candles in total from 2021-05-06T02:20:00.000Z to 2021-11-30T10:19:00.000Z
400000 XRP/USDT candles in total from 2021-02-25T17:10:00.000Z to 2021-11-30T10:19:00.000Z
500000 XRP/USDT candles in total from 2020-12-18T06:40:00.000Z to 2021-11-30T10:19:00.000Z
600000 XRP/USDT candles in total from 2020-10-09T20:00:00.000Z to 2021-11-30T10:19:00.000Z
700000 XRP/USDT candles in total from 2020-08-01T09:20:00.000Z to 2021-11-30T10:19:00.000Z
800000 XRP/USDT candles in total from 2020-05-23T22:40:00.000Z to 2021-11-30T10:19:00.000Z
900000 XRP/USDT candles in total from 2020-03-15T12:00:00.000Z to 2021-11-30T10:19:00.000Z
1000000 XRP/USDT candles in total from 2020-01-06T06:50:00.000Z to 2021-11-30T10:19:00.000Z
1100000 XRP/USDT candles in total from 2019-10-28T21:40:00.000Z to 2021-11-30T10:19:00.00

In [178]:
dat = pd.DataFrame(all_ohlcv, columns=['time','open','high','low','close','volume'])
dat['time'] = [datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S') for t in dat['time']]
dat['time'] = pd.to_datetime(dat['time'])

# dat.to_csv('XRP-USDT_1min.csv', index=False)

환율데이터 불러와서 저장

In [65]:
# # pip install finance-datareader 패키지 이름 주의

# import FinanceDataReader as fdr


# ex_rate = fdr.DataReader('USD/KRW','2018')
# ex_rate = ex_rate.reset_index()[['Date','Close']]
# ex_rate['Date'] = ex_rate['Date'].astype('str')
# ex_rate.head()

# # index 사용해서 전처리가 용이
# ex_rate = ex_rate.set_index(['Date'])
# ex_rate.index = pd.DatetimeIndex(ex_rate.index)

# # 공휴일 빈날짜까지 다 채워서 환율 만들기
# idx = pd.date_range(ex_rate.index[0],ex_rate.index[-1])
# ex_rate = ex_rate.reindex(idx)

# # 금요일값으로 토일 채우기 : 밀어채우기
# ex_rate = ex_rate.fillna(method='pad')
# ex_rate = ex_rate.reset_index().rename(columns={"index": "Date"})

# # 데이터 하루 뒤로 만들어서 환율 오늘 종가를 내일데이터랑 결합
# ex_rate['Date'] = ex_rate['Date'] + timedelta(days=1)
# ex_rate['Date'] = ex_rate['Date'].astype('str')

### 환율데이터 전처리

In [1]:
import os
import pandas as pd
path_dir = './환율변동조회/'
file_list = os.listdir(path_dir)
file_list = [path_dir + a for a in file_list]
# pd.read_csv(file_list[0])
file_list[0]

'./환율변동조회/환율변동조회_20211128 (100).txt'

In [2]:
result = []
for i in range(len(file_list)):
    f = open(file_list[i])
    lines = f.readlines()
    ymd = lines[2].split()[2]
    clean_lines = [a.split() for a in lines[3:]]
    dat = pd.DataFrame(clean_lines[1:], columns=clean_lines[0])
    dat  = dat[['시간','매매기준율']]
    dat['시간'] = ymd+' '+dat['시간']
    result.append(dat)

In [11]:
ex_rate = pd.concat(result)
ex_rate = ex_rate.rename(columns={'시간':'date','매매기준율':'USD/KRW'}).sort_values('date')\
    .drop_duplicates().reset_index(drop=True)


ex_rate['date'] = ex_rate['date'].str.slice_replace(start=-2, repl='00') # 초없애기
ex_rate['USD/KRW'] = ex_rate['USD/KRW'].str.replace(',','').astype('float') # 1,234.5 같은 형태의 캐릭터라 쉼표 없애기

ex_rate = ex_rate.groupby(ex_rate['date']).mean() # 초없에서 생기는 같은 시간 평균으로 하나만들기
ex_rate.index = pd.DatetimeIndex(ex_rate.index)

# 1분단위로 다 채워서 조인할수있게 만들기
idx = pd.date_range(ex_rate.index[0],ex_rate.index[-1] ,freq='1min')
ex_rate = ex_rate.reindex(idx)
ex_rate = ex_rate.fillna(method='pad')

In [12]:
ex_rate.reset_index().rename(columns={'index':'date'}).to_csv('exchange_rate_1min.csv',index=False)

In [180]:
dat = pd.merge(dat,ex_rate, left_on='time', right_index=True, how='left')

for i in ['open','high','low','close']:
    dat[i] = dat[i] * dat['USD/KRW']

In [181]:
dat

,time,open,high,low,close,volume,USD/KRW
0,2018-05-04 17:11:00,538.250000,538.250000,538.250000,538.250000,109.51,1076.5
1,2018-05-04 17:12:00,968.850000,1001.134235,915.025000,968.850000,167543.47,1076.5
2,2018-05-04 17:13:00,968.850000,1001.145000,958.095765,979.615000,179750.59,1076.5
3,2018-05-04 17:14:00,979.615000,1614.750000,968.946885,1001.134235,107222.48,1076.5
4,2018-05-04 17:15:00,1001.134235,1022.685765,979.830300,979.830300,171304.56,1076.5
...,...,...,...,...,...,...,...
1881995,2021-11-30 19:15:00,NaN,NaN,NaN,NaN,118690.00,NaN
1881996,2021-11-30 19:16:00,NaN,NaN,NaN,NaN,129497.00,NaN
1881997,2021-11-30 19:17:00,NaN,NaN,NaN,NaN,92410.00,NaN
1881998,2021-11-30 19:18:00,NaN,NaN,NaN,NaN,50187.00,NaN


In [182]:
dat = dat.drop(['USD/KRW'],axis=1).rename(columns={'time':'date'})
dat = dat[1:].drop_duplicates().dropna()
dat.to_csv('XRP-Binance-USDT_1min.csv', index=False)

In [17]:
# from datetime import datetime, timedelta

# dat['time'] = pd.to_datetime(dat['time'])
# # yyyy-mm-dd 형식 변수 만들어주기
# dat['ymd'] = dat['time'].dt.date.astype('str')



# dfs = pd.DataFrame()
# for df in dat.groupby(['ymd']):
#     # df[0]:날짜, df[1]:dateframe
#     USD = ex_rate.loc[ex_rate['Date']==df[0]]['Close']
#     try:
#         df[1].loc[:, ['open','high','low','close']] *= float(USD)
#         dfs = dfs.append(df[1])
#     except:
#         continue
#     # print(USD)
# dat = dfs[1:].set_index(['time']).drop_duplicates()


# dat = dat.drop(['ymd'],axis=1).reset_index()
# dat.rename(columns={'time':'date'}).to_csv('XRP-Binance-USDT_1min.csv', index=False)

In [186]:
import math
import pyupbit
import time
def read_min_price(ticker, count):
    """업비트에서 1분봉 코인시세를 200개씩 읽어서 count개로 합친 데이터프레임 반환"""
    date = None
    dfs = pd.DataFrame()
    for i in range(math.ceil(count / 200)):
        df = pyupbit.get_ohlcv(ticker, interval="minute1", to=date)
        dfs = dfs.append(df)
        try:
            date = df.index[0]
        except IndexError:
            break
        time.sleep(0.101) #요청 수 제한
        if i % 500 == 499:
            print(f"[{ticker}] CONCATENATING {len(dfs)} rows...")
    dfs["date"] = dfs.index
    return dfs

In [187]:
XRP_KRW = read_min_price('KRW-XRP',365*5*60*24)

[KRW-XRP] CONCATENATING 100000 rows...
[KRW-XRP] CONCATENATING 200000 rows...
[KRW-XRP] CONCATENATING 300000 rows...
[KRW-XRP] CONCATENATING 400000 rows...
[KRW-XRP] CONCATENATING 500000 rows...
[KRW-XRP] CONCATENATING 600000 rows...
[KRW-XRP] CONCATENATING 700000 rows...
[KRW-XRP] CONCATENATING 800000 rows...
[KRW-XRP] CONCATENATING 900000 rows...
[KRW-XRP] CONCATENATING 1000000 rows...
[KRW-XRP] CONCATENATING 1100000 rows...
[KRW-XRP] CONCATENATING 1200000 rows...
[KRW-XRP] CONCATENATING 1300000 rows...
[KRW-XRP] CONCATENATING 1400000 rows...
[KRW-XRP] CONCATENATING 1500000 rows...
[KRW-XRP] CONCATENATING 1600000 rows...
[KRW-XRP] CONCATENATING 1700000 rows...
[KRW-XRP] CONCATENATING 1800000 rows...
[KRW-XRP] CONCATENATING 1900000 rows...
[KRW-XRP] CONCATENATING 2000000 rows...


In [89]:
XRP_KRW.to_csv('XRP-Upbit-KRW_1min.csv',index=False)

In [188]:
XRP_KRW

,open,high,low,close,volume,date
2021-11-30 16:16:00,1230.0,1235.0,1230.0,1235.0,26639.892726,2021-11-30 16:16:00
2021-11-30 16:17:00,1230.0,1235.0,1230.0,1230.0,59745.383276,2021-11-30 16:17:00
2021-11-30 16:18:00,1230.0,1235.0,1230.0,1235.0,34790.441021,2021-11-30 16:18:00
2021-11-30 16:19:00,1235.0,1235.0,1230.0,1235.0,31388.078505,2021-11-30 16:19:00
2021-11-30 16:20:00,1230.0,1235.0,1230.0,1230.0,127212.649080,2021-11-30 16:20:00
...,...,...,...,...,...,...
2017-09-26 05:22:00,203.0,203.0,203.0,203.0,0.000100,2017-09-26 05:22:00
2017-09-26 05:24:00,204.0,204.0,204.0,204.0,0.000100,2017-09-26 05:24:00
2017-09-26 05:27:00,203.0,203.0,203.0,203.0,0.000100,2017-09-26 05:27:00
2017-09-26 05:30:00,204.0,204.0,204.0,204.0,0.000100,2017-09-26 05:30:00


In [185]:
# upbit = ccxt.upbit()
# all_ohlcv = scrape_ohlcv(upbit, 3, 'KRW-XRP', '1m', '2017-01-01 00:00:00Z', 1000)

In [51]:
def make_candle(df,interval = 10, open_time: str = None ):
    
    offset = timedelta(minutes = interval)
    if open_time is None:
        start_date = df.index[0]
    else: 
        start_date = df.index[0].replace(hour = int(open_time[:2]), minute = int(open_time[3:]))
    end_date = df.index[-1]
    counts = (pd.to_datetime(end_date)-pd.to_datetime(start_date)) // offset + 1
    rows = []
    for i in range(1,counts):
        reduced_df = df.loc[start_date + i * offset : start_date + (i+1)*offset - timedelta(minutes = 1)]
        try :#예전자료에는 없는 분봉들이 있음
            date = start_date + i * offset
            open = reduced_df['open'][0]
            high = reduced_df['high'].max()
            low = reduced_df['low'].min()
            close = reduced_df['close'][-1]
            volume = reduced_df['volume'].sum()
            rows.append((date, open, high,low,close,volume))
        except IndexError:
            continue
    data = pd.DataFrame(rows, columns = ['date', 'open', 'high','low','close', 'volume'])
    data.index = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M')
    data = data.drop("date", axis =1 )
    return data